<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/good_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.1/190.1 KB 4.7 MB/s eta 0:00:00


In [2]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [5]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99]):
      self.period = period
      self.increased = increased
      self.traded = traded

      self.price_dic = {}

      self.start, self.today = self.get_date()
      self.df = self.get_stock_df()
      self.filtered_df = self.get_filtered_df()
      self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_result_df(self):
    for i in self.filtered_df['종목코드']:
      self.price_dic[i] = fdr.DataReader(str(i), self.start, self.today)

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    return self.filtered_df.sort_values('Sharpe', ascending=False)

  def plot_df(self, start_from=0, to=5):
    fig = go.Figure()
    for i in self.price_dic:
      if (i in self.result_df['종목코드'][start_from:to].values):
        fig.add_trace(go.Scatter(x= self.price_dic[i].index, \
                                 y= self.price_dic[i]['Close']/self.price_dic[i]['Close'][0], \
                                 name= self.filtered_df[self.filtered_df['종목코드']==i]['종목명'].item()))

    fig.update_layout(title=self.today,
                      xaxis_title='Date',
                      yaxis_title='Return (%)')
    fig.show()

  def plot_stock(self, id):
    t = stocklist.price_dic[id]
    candlesticks = go.Candlestick(
        x=t.index,
        open=t['Open'],
        high=t['High'],
        low=t['Low'],
        close=t['Close'],
        showlegend=False
    )

    volume_bars = go.Bar(
        x=t.index,
        y=t['Volume'],
        showlegend=False,
        marker={
            "color": "rgba(128,128,128,0.5)",
        }
    )

    fig = go.Figure(candlesticks)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(candlesticks, secondary_y=False
                  )
    fig.add_trace(volume_bars, secondary_y=True)
    fig.update_layout(title= self.filtered_df[self.filtered_df['종목코드']==id]['종목명'].item(), height=800)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        ]
    )
    fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
    fig.update_yaxes(title="Volume $", secondary_y=False, showgrid=False)
    fig.show()

In [6]:
highest = [0.99, 1]
higher = [0.9, 0.99]
longer = 500

In [7]:
stocklist = StockList()
stocklist_highest = StockList(250, highest, highest)
stocklist_long = StockList(longer)

In [8]:
stocklist_highest.result_df

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금,Sharpe,Sortino
1765,086520,에코프로,90236,722000,631764,700.12,178365583,41620087691400,19.203038,51.007680
1274,277810,레인보우로보틱스,29300,114800,85500,291.81,243238120,17117481127000,16.914490,40.868262
1341,095500,미래나노텍,8170,32650,24480,299.63,1018915722,19350219073820,15.618335,30.834965
203,001570,금양,9110,89500,80390,882.44,1039898349,27888389025380,14.700472,26.575424


In [9]:
stocklist_highest.plot_df()

In [10]:
stocklist.result_df

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금,Sharpe,Sortino
2323,022100,포스코 ICT,5750,10770,5020,87.30,390084919,3349676994620,31.722802,73.925156
1273,281740,레이크머티리얼즈,5000,8870,3870,77.40,328013472,2333264691415,28.051256,56.323247
1315,086900,메디톡스,107280,254500,147220,137.23,15307726,2818340578000,26.806096,56.114993
1082,121600,나노신소재,84100,184800,100700,119.74,62925023,7363362559100,26.046857,50.026293
1276,090360,로보스타,20950,37550,16600,79.24,111371731,3731973121900,24.193434,52.008212
1810,037950,엘컴텍,1190,2155,965,81.09,1581289279,2621417745249,23.177848,52.869789
1445,000250,삼천당제약,34600,87500,52900,152.89,67072114,3578553215000,22.466956,51.617119
165,002710,TCC스틸,13950,37300,23350,167.38,134315820,2251630192140,22.292262,47.741866
1730,058610,에스피지,16150,32200,16050,99.38,314212162,7657338400220,22.066645,43.075493
744,014580,태경비케이,4395,8910,4515,102.73,312981368,2172002718285,21.824771,41.060321


In [11]:
stocklist.plot_df()

In [12]:
stocklist_long.result_df

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금,Sharpe,Sortino
1804,048260,오스템임플란트,100650,187900,87250,86.69,55909768,7331115124179,35.996335,60.100334
859,012450,한화에어로스페이스,44400,117300,72900,164.19,296328093,19941693153761,33.204903,50.515183
622,007660,이수페타시스,4500,10580,6080,135.11,918075089,7225450023115,26.265372,55.092434
1066,121600,나노신소재,60500,184800,124300,205.45,118315175,11237908405900,26.222760,49.083705
1410,000250,삼천당제약,45200,87500,42300,93.58,90540117,4494677469900,26.121989,55.315932
1684,058610,에스피지,10300,32200,21900,212.62,440551616,9407330299541,25.255531,46.587834
1252,090360,로보스타,20150,37550,17400,86.35,179968790,5512969071450,25.189356,48.802634
433,006110,삼아알미늄,35300,80200,44900,127.20,122153470,4473436743200,24.609231,46.138696
1133,290670,대보마그네틱,32400,82700,50300,155.25,74519820,5082756246900,23.268972,47.547926
879,322000,현대에너지솔루션,20800,51300,30500,146.63,123784437,5946365013900,22.895309,43.099177


In [13]:
stocklist_long.plot_df()